In [15]:
import numpy as np 
import random
from scipy.optimize import minimize

# generate training and tesing data
Nt = 2
Nr = 4
# generate channel
H = np.sqrt(1/2)*(np.random.randn(Nr,Nt)+1j*np.random.randn(Nr,Nt))

In [24]:
# generate transmit signal
def generate_random_bit_sequence(length):
    return ''.join(random.choice('01') for _ in range(length))

def qam16_modulation(binary_input):
    mapping = {
        '0000': (1+1j),
        '0001': (1+3j),
        '0010': (3+1j),
        '0011': (3+3j),
        '0100': (1-1j),
        '0101': (1-3j),
        '0110': (3-1j),
        '0111': (3-3j),
        '1000': (-1+1j),
        '1001': (-1+3j),
        '1010': (-3+1j),
        '1011': (-3+3j),
        '1100': (-1-1j),
        '1101': (-1-3j),
        '1110': (-3-1j),
        '1111': (-3-3j)
    }
    return mapping.get(binary_input, "Invalid binary input")/np.sqrt(10)

def generate_x_sequence(length, Nt):
    total_bits_sequence = generate_random_bit_sequence(length*Nt*4)
    bits_sequence = [total_bits_sequence[i:i+4] for i in range(0, len(total_bits_sequence), 4)]
    x_sequence = [np.array([qam16_modulation(bits_sequence[i+j]) for j in range(Nt)]) for i in range(0, len(bits_sequence), Nt)]
    return bits_sequence, x_sequence

# noise
SNR_dB = 0
SNR = 10.0**(SNR_dB/10.0)
def generate_noise(SNR, Nr):
    return np.sqrt(1/(2*SNR))*(np.random.randn(Nr,1)+1j*np.random.randn(Nr,1))

# generate training and tesing data
def generate_data(Nr,Nt,SNR,length,H_channel):
    bits_sequence, x_sequence = generate_x_sequence(length, Nt)
    n_sequence = [generate_noise(SNR, Nr) for i in range(length)]
    y_sequence = [np.dot(H_channel, x_sequence[i].reshape(Nt,1)) + n_sequence[i] for i in range(length)]
    return bits_sequence, x_sequence, y_sequence

training_length = 16
bits_sequence, x_sequence, y_sequence = generate_data(Nr,Nt,SNR,training_length,H)

In [25]:
def bits2signals(bits):
    # bits: input binary string with length of (4*Nt) 
    return np.array([qam16_modulation(bits[i:i+4]) for i in range(0, len(bits), 4)]).reshape(Nt,1)
def calculate_layer1(H_hat, y):
    dimension_layer1 = 2**(4*Nt)
    output = {}
    for index in range(dimension_layer1):
        bits = str(bin(index)[2:].zfill(4*Nt))
        s = bits2signals(bits)
        error = y - np.dot(H_hat,s)
        value =  np.exp(-np.square(np.linalg.norm(error)))
        output[bits] = value
    return output

def calculate_layer2(layer1_output):
    sum_exp = [[0 for i in range(2)] for j in range(4*Nt)]
    for bits in layer1_output:
        value = layer1_output[bits]
        for index in range(4*Nt):
            sum_exp[index][eval(bits[index])] += value
    output = {}
    for index in range(4*Nt):
        # llr = np.log(sum_exp[index][1]/sum_exp[index][0])
        output[index] = (sum_exp[index][1])/(sum_exp[index][1]+sum_exp[index][0])
    return output

def calculate_cross_entropy(layer2_output, true_sequence):
    dimension = len(true_sequence)
    entropy = 0
    for index in range(dimension):
        if true_sequence[index] == '1':
            entropy += (-np.log(layer2_output[index]))
    return entropy

def calculate_square_error(layer2_output, true_sequence):
    dimension = len(true_sequence)
    loss = 0
    for index in range(dimension):
        if true_sequence[index] == '1':
            loss += np.square(1-layer2_output[index])
        else:
            loss += np.square(layer2_output[index])
    return loss

def calculate_cost_function(H_hat_vec):
    H_hat = H_hat_vec[0:Nr*Nt].reshape(Nr,Nt)+1j*H_hat_vec[Nr*Nt:2*Nr*Nt].reshape(Nr,Nt)
    # H_hat = H_hat_vec
    total_loss = 0
    for ii in range(training_length):
        layer1_output = calculate_layer1(H_hat, y_sequence[ii])
        layer2_output = calculate_layer2(layer1_output)
        true_sequence = ''.join(bits_sequence[ii*Nt+jj] for jj in range(Nt))
        total_loss += calculate_square_error(layer2_output,true_sequence)
    mean_loss = total_loss/training_length
    print(mean_loss)
    return mean_loss
        
# calculate_cost_function(H)

In [26]:
def detection(y, H_trained):
    layer1_output = calculate_layer1(H_trained, y)
    layer2_output = calculate_layer2(layer1_output)
    detect_result = ''
    for ii in range(len(layer2_output)):
        if(layer2_output[ii]>0.5):
            detect_result += '1'
        else:
            detect_result += '0'
    return(detect_result)

def count_differences(str1, str2):
    return sum(a != b for a, b in zip(str1, str2))

def calculate_BER(H_trained):
    # tesing set
    testing_length = 1000
    bits_sequence_testing, x_sequence_testing, y_sequence_testing = generate_data(Nr,Nt,SNR,testing_length,H)
    error = 0
    for ii in range(len(y_sequence_testing)):
        detect_result = detection(y_sequence_testing[ii], H_trained)
        true_sequence = ''.join(bits_sequence_testing[ii*Nt+jj] for jj in range(Nt))
        error += count_differences(detect_result, true_sequence)
    BER = error/(len(y_sequence_testing)*len(detect_result))
    return BER

In [29]:
def training_testing():
    H_hat_vec = np.sqrt(1/2)*(np.random.randn(Nr*Nt*2))
    BER_performance = 1

    out = minimize(calculate_cost_function, x0=H_hat_vec, method="COBYLA", options={'maxiter':500})

    H_hat_vec = out.x

    H_trained = H_hat_vec[0:Nr*Nt].reshape(Nr,Nt)+1j*H_hat_vec[Nr*Nt:2*Nr*Nt].reshape(Nr,Nt)
    BER_performance = calculate_BER(H_trained)

    return H_trained, BER_performance

training_testing()


3.01427566228821
3.155523145625767
2.9959791818769874
3.308101677840054
3.187016007811539
3.2222492794902964
2.863934210426205
3.1447101666815898
2.9516961249761633
2.7572114597434716
2.855480988364436
2.9647917490631794
2.437959982231955
2.497915294751345
2.4531557312354733
2.5932813250932303
2.4855936878361797
2.2990450312154485
2.394026568619216
2.1761908717072447
2.3440307498813255
2.3211463658356855
2.099321954344559
2.2102328267476756
2.38463590307817
2.169163330283286
2.2738978792880804
2.1493710324522
2.3489884592283894
2.105098050727789
2.262830844909533
2.107885989981725
2.0403353413385714
2.0240091519229986
2.0133669462357573
2.0594112540378027
2.11323507584585
2.061298480144549
2.0284046049734537
2.0539963654931284
2.0598698814701337
1.9356309855399672
1.9256408236814018
1.8983288613707643
1.8935802990036024
1.9062158885573854
2.108413183487791
1.9347204986657607
1.9038437286992327
1.8917021866599901
1.9410552454354277
1.9394049990383495
1.9109960668677939
1.868971015749752

(array([[ 0.35774276+1.22189288j, -0.86149197+1.10219513j],
        [ 0.88575085-0.37516382j, -0.29658438+0.68993921j],
        [-0.27069744-0.02968713j, -0.6219367 -0.11387994j],
        [-0.78843526-0.30802891j, -0.25192007+0.96659435j]]),
 0.21225)

In [28]:
H

array([[ 0.4052545 +1.1364205j , -0.69477697+0.67637798j],
       [ 0.51651071-0.71077743j, -0.25098806+0.67218313j],
       [-0.25288016+0.6460937j , -0.20178888+0.01503802j],
       [-0.87851524-0.21199186j, -0.59884798+0.84684823j]])

In [54]:
from sphere_decoding.sphereDecodingUseC import sphere_decoding_BER
bits_sequence, x_sequence, y_sequence = generate_data(Nr,Nt,20,1024,H)
a = sphere_decoding_BER(H, y_sequence, bits_sequence, 1)
print("SD: "+str(a))

SD: 0.0108642578125
